# Evaluation of Customer Switching Behavior between Prepaid Mobile Plans

### Abstract

One new trending feature of Smartphones is the support for E-SIM (Embedded Subscriber Identification Module) cards. These allow the user to simultaneously subscribe to multiple cellular providers while also supporting at most one physical SIM (Subscriber Identification Module) card. This feature allows customers to easily switch between providers and is especially useful for those who use prepaid plans which are popular in developing countries. A customer may have multiple providers and, at any point in time, can choose the provider with the most cost effective data plan. This means that cellular providers must consider soft-churn, where the consumer dynamically switches between multiple plans from multiple providers, in addition to the more traditional churn, where a consumer switches providers. This means that data pricing for such consumers must now be more personalized in order to be competitive and maximize profits. We determine the optimal personalized prepaid plan for such users, while providing a competitive advantage to the provider. Examples are provided to demonstrate the benefit, and numerical results corroborate our premise that personalized over traditional pricing plans can improve the revenue of the provider.

This notebook has the code developed to evaluate the switching behavior of customers, and produce the graphs shown under the Numerical Results section of the paper.

## Necessary Libraries 

In [ ]:
%pylab inline
import numpy as np
import pandas as pd
import category_encoders as ce
import seaborn as sns
from datetime import date, datetime
import matplotlib.pyplot as plt
import re #For Regex Operations

from scipy.stats.kde import gaussian_kde
from scipy.stats import norm
from scipy.interpolate import make_interp_spline, BSpline

## Read in CSV

In [ ]:
# Relevant Fields Description
# Note that some fields were removed to preserve anonymity of data
'BUNDLEID'             # Prepaid Plan Identifier - Hashed to preserve anonymity
'Duration'             # Given Length of the Plan
'INITIALBALANCELIMIT'  # Initial amount of data allowed
'Time_Taken'           # Amount in days customer took...
'Usage'                # Amount of data in Mb that the customer used at the end
'Bundle_Status'        # If the bundle was expired or exhausted

data_usage_eval = pd.read_csv('data_usage.csv')
data_usage_eval.head()

## Functions

### Obtain Fraction X of Data Used for all users

In [ ]:
def obtain_x(data_usage_eval):
    '''
    Calculates X for expired and exhausted prepaid plans for all users.
    If the plan is expired, X is the fraction of data used before expiration.
    If the plan is exhausted, X is the fraction of time taken over the duration.

    :param data_usage_eval: Accepts the dataframe with the relevant data and fields. 
    :return: data_usage_eval: Dataframe with updated fields.
    '''
    
    # Insert Column X into the datafram
    # If headers do not exist, insert them   
    if not 'X' in data_usage_eval.columns:
        data_usage_eval.insert(loc=6, column='X', value=' ')

    for i in range(len(data_usage_eval)):
        if data_usage_eval.at[i, 'Bundle_Status'] == 'Bundle_Expired':
            data_usage_eval.at[i, 'X'] = data_usage_eval.at[i, 'Usage'] / data_usage_eval.at[i, 'INITIALBALANCELIMIT']
        elif data_usage_eval.at[i, 'Bundle_Status'] == 'Bundle_Exhausted':
            data_usage_eval.at[i, 'X'] = data_usage_eval.at[i, 'Time_Taken'] / data_usage_eval.at[i, 'Duration']

    data_usage_eval['X'] = pd.to_numeric(data_usage_eval['X'])
    
    return data_usage_eval

### Graph displaying the fraction of data used before expiration of a plan

In [ ]:
def expired_x_graph_display(data_usage_eval):
    '''
    Displays the graph for the fraction of data used before expiration, 
    against the different plans from the provider.

    :param data_usage_eval: Accepts the dataframe with the relevant data and fields. 
    :output: Shows the graph
    '''
    exp_prepaid = data_usage_eval.loc[data_usage_eval['Bundle_Status'] == 'Bundle_Expired']
    group_prepaid = exp_prepaid.groupby(['BUNDLEID']).size().reset_index(name='counts')
    count = group_prepaid['counts']
    group_prepaid_mean = exp_prepaid.groupby(['BUNDLEID']).mean().reset_index()
    group_prepaid_mean.insert(loc=6, column='Counts', value=count)

    sns.set_style('darkgrid')
    sns.set(rc={'figure.figsize':(11.7,8.27)})

    x = group_prepaid_mean.index
    y = group_prepaid_mean['X']

    ax = sns.barplot(x=x, y=y, data=group_prepaid_mean, palette="Blues_d")
    ax.set(xlabel='Different Prepaid Plans ', ylabel='Fraction of data used before expiration (x)')
    plt.show()

### Calculate Profits for Personalised and Traditional for different profit margins

In [ ]:
def get_profit(kp_list, data_usage_eval):
    '''
    Obtains the profit for traditional and personalised plans based
    on the profit margin

    :param data_usage_eval: Accepts the dataframe with the relevant data and fields. 
    :output: Saves the profits for exhausted and expired plans for both traditional and personalised, 
    this is done to calculate the profit ratios later on.
    '''
    
    exp_personalised = dict()
    exp_traditional = dict()
    exh_personalised = dict()
    exh_traditional = dict()

    # If headers do not exist, insert them    
    if not set(['Personalised_Profit','Traditional_Profit']).issubset(data_usage_eval.columns):
        data_usage_eval.insert(loc=7, column='Personalised_Profit', value=' ')
        data_usage_eval.insert(loc=8, column='Traditional_Profit', value=' ')

    for kp in kp_list:
        print("Profit Margin: ", kp)
        margin = 1 / (1+kp)

        for i in range(len(data_usage_eval)):
            if data_usage_eval.at[i, 'X'] <= margin:
                data_usage_eval.at[i, 'Personalised_Profit'] = np.multiply(data_usage_eval.at[i, 'X'], kp)
                data_usage_eval.at[i, 'Traditional_Profit'] = 0
            elif data_usage_eval.at[i, 'X'] > margin:
                data_usage_eval.at[i, 'Traditional_Profit'] = 1 - data_usage_eval.at[i, 'X']
                data_usage_eval.at[i, 'Personalised_Profit'] = 0

        data_usage_eval['Personalised_Profit'] = pd.to_numeric(data_usage_eval['Personalised_Profit'], errors='coerce')
        data_usage_eval['Traditional_Profit'] = pd.to_numeric(data_usage_eval['Traditional_Profit'], errors='coerce')

        expired = data_usage_eval.loc[data_usage_eval['Bundle_Status'] == 'Bundle_Expired']
        exhausted = data_usage_eval.loc[data_usage_eval['Bundle_Status'] == 'Bundle_Exhausted']
        
        exp_personalised[kp] = expired['Personalised_Profit'].sum()
        exp_traditional[kp] = expired['Traditional_Profit'].sum()
        exh_personalised[kp] = exhausted['Personalised_Profit'].sum()
        exh_traditional[kp] = exhausted['Traditional_Profit'].sum()
        
        print("Expired Personalized Profit: ", exp_personalised[kp])
        print("Expired Traditional Profit: ", exp_traditional[kp])
        print("Exhausted Personalized Profit: ", exh_personalised[kp])
        print("Exhausted Traditional Profit: ", exh_traditional[kp])
        print("\n\n")

    return exp_personalised, exp_traditional, exh_personalised, exh_traditional

### Get Profit Ratios 

In [ ]:
def get_profit_ratios(kp_list, exp_personalised, exp_traditional, exh_personalised, exh_traditional):
    '''
    Calculates the profit ratios using the personalized and 
    traditonal profits previously calculated.

    :param kp_list: Range of profit margins from 0.02 - 0.15
    :param exp_personalised: Personalized profits for the range of profit margins [0.02 - 0.15] for expired fields. 
    :param exp_traditional: Traditional profits for the range of profit margins [0.02 - 0.15] for exhausted fields. 
    :param exh_personalised: Personalized profits for the range of profit margins [0.02 - 0.15] for expired fields. 
    :param exh_traditional: Traditional profits for the range of profit margins [0.02 - 0.15] for exhausted fields.
    
    :return: exp_ratios: Expired Profit Ratios.
    :return: exp_pers_profits: Personalised Profits for Expired Plans.
    :return: exp_trad_profits: Traditional Profits for Expired Plans.
    :return: exh_ratios: Exhausted Profit Ratios.
    :return: exh_pers_profits: Personalised Profits for Exhausted Plans.
    :return: exh_trad_profits: Traditional Profits for Exhausted Plans.
    '''
    exp_pers_profits = []
    exp_trad_profits = []
    exh_pers_profits = []
    exh_trad_profits = []
    exp_ratio = []
    exh_ratio = []

    for x in kp_list:
        exp_pers_profits.append(exp_personalised[x])
        exp_trad_profits.append(exp_traditional[x])
        exh_pers_profits.append(exh_personalised[x])
        exh_trad_profits.append(exh_traditional[x])

        exp_ratio.append(exp_personalised[x]/exp_traditional[x])
        exh_ratio.append(exh_personalised[x]/exh_traditional[x])
        
    # Expired Data
    exp_ratios = np.array(exp_ratio)
    exp_pers_profits = np.array(exp_pers_profits)
    exp_trad_profits = np.array(exp_trad_profits)

    # Exhausted Data
    exh_ratios = np.array(exh_ratio)
    exh_pers_profits = np.array(exh_pers_profits)
    exh_trad_profits = np.array(exh_trad_profits)
        
    return exp_ratios, exp_pers_profits, exp_trad_profits, exh_ratios, exh_pers_profits, exh_trad_profits

### Produce Profit Ratio Graphs

In [ ]:
def profit_ratio_graph(kp_list, ratio, personalised, traditional):
    '''
    Calculates the profit ratios using the personalized and 
    traditonal profits previously calculated.

    :param kp_list: Range of profit margins from 0.02 - 0.15
    :param ratio: Profit Ratios Previously Calculated
    :param personalised: 
    :param traditional: 
    
    :output: Profit Ratio Graph

    '''
    # Profit Margins     
    kp = np.array(kp_list)
    # Range of profit margin from min to max
    xnew = np.linspace(kp.min(), kp.max(), 3000)
    
    # Profit Ratios
    a_BSpline = make_interp_spline(kp, ratio)
    # Personalised Profits
    pers_BSpline = make_interp_spline(kp, personalised)
    # Traditional Profits
    trad_BSpline = make_interp_spline(kp, traditional)
    
    y_new = a_BSpline(xnew)
    y_pers = pers_BSpline(xnew)
    y_trad = trad_BSpline(xnew)

    # Set the x axis label
    plt.xlabel('Profit Margin (kp)')
    # Set the y axis label
    plt.ylabel('Profit Ratio (G)')
    
    # Plot the graph
    plt.plot(xnew, y_new)
    plt.show()

## Call All Functions Here

In [ ]:
# Get X
data_usage_eval = obtain_x(data_usage_eval)
data_usage_eval.head()

In [ ]:
# Display Graph for expired plans
expired_x_graph_display(data_usage_eval)

### Store New Data to Calculate Profit Ratios

In [ ]:
# Save profits for personalised and traditional plans
# kp for 0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.15
kp_list = [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.15]
exp_personalised, exp_traditional, exh_personalised, exh_traditional = get_profit(kp_list, data_usage_eval)

### Get Profit Ratios 

In [ ]:
exp_ratios, exp_pers_profits, exp_trad_profits, exh_ratios, exh_pers_profits, exh_trad_profits = get_profit_ratios(kp_list, exp_personalised, exp_traditional, exh_personalised, exh_traditional)

### Produce Graphs

In [ ]:
# Profit Ratio Graph for Expired Plans
profit_ratio_graph(kp_list, exp_ratios, exp_pers_profits, exp_trad_profits)

In [ ]:
# Profit Ratio Graph for Exhausted Plans
profit_ratio_graph(kp_list, exh_ratios, exh_pers_profits, exh_trad_profits)